In [ ]:
import netCDF4 as nc
import numpy as np
from scipy.interpolate import interp1d
import xarray as xr
import xroms

fnumbers = [f'{num:05}' for num in range(1461, 10641+1, 30)] # last valid file 10641

for fnumber in fnumbers:
    print(fnumber)
    fname = '/srv/scratch/z3533156/26year_BRAN2020/outer_avg_'+str(fnumber)+'.nc'
    dataset = nc.Dataset(fname)
    z_r = np.load('/srv/scratch/z5297792/z_r.npy')
    z_r = np.transpose(z_r, (1, 2, 0))

    ds = xr.open_dataset(fname)
    N2 = ds.xroms.N2
    N2 = np.transpose(N2.data, axes=(3, 2, 1, 0))
    N2 = np.flip(N2, axis=2)

    N2 = N2[:,:,1:,:].squeeze()
    depths = z_r[150,150,:]

    def depth_interpolator(M,z_r,depths):

        # Dimensions of M and z_r
        dim_i, dim_j, dim_sigma, dim_time = M.shape
        assert z_r.shape == (dim_i, dim_j, dim_sigma)

        # Initialize interpolated matrix
        M_interpolated = np.zeros((dim_i, dim_j, len(depths), dim_time))

        # Interpolate for each day (time dimension)
        for t in range(dim_time):
            for i in range(dim_i):
                for j in range(dim_j):
                    # Get depth levels from z_r
                    depth_levels = z_r[i, j, :]
                    
                    # Get values of M at each depth level for current time t
                    values = M[i, j, :, t]
                    
                    # Perform interpolation
                    if np.all(np.isnan(values)):
                        # Handle case where all values are NaN (e.g., outside valid data range)
                        interpolated_values = np.full(len(depths), np.nan)
                    else:
                        # Use linear interpolation
                        interpolator = interp1d(depth_levels, values, kind='linear', bounds_error=False, fill_value=np.nan)
                        interpolated_values = interpolator(depths)
                    
                    # Assign interpolated values to M_interpolated for current time t
                    M_interpolated[i, j, :, t] = interpolated_values
            M_interpolated[:,:, 0, t] = M[:,:,0,t]

        return M_interpolated

    N2 = depth_interpolator(N2, z_r, depths)

    np.save('/srv/scratch/z5297792/Climatology/N2s/N2s/N2_'+str(fnumber)+'.npy', N2)
    
    del N2
    

01461
